**The first thing to do in this case is to get the necessary libraries and modules to use it in python.**

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import os
from urllib.parse import urljoin
import pandas as pd

**After a long research i could find a solution to apply the scrapper. In this case i had to bring up the pdf files of the Audit Reports Resolution of every year. Since the information is stored by year and by Unit Auditor: "Fondos propios" and "Auditoría Interna".**

In [22]:
"""" EXTRACT PDF FOR AUDITORIAS 2018
IN THIS CASE I STARTED WITH 2018 YEAR AND 'FONDOS PROPIOS' SECTION """


for page in range(1,11): # WITH IT INDICATE THE NUMBER OF PAGES TO LOOK FOR THE FILES
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start='
    
    folder_location = r'/Users/leona/Documents/Python/Contraloria/2018' # It creates a folder to save files
    #If there is no such folder, the script will create one automatically
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url+str((page - 1) * 15))
    """"response = requests.get(url) - with this, it indicates the number of pages, 
    but to make it automatic, it adds "*15" because every next page the link adds 15"""
    soup= bs(response.text, "html.parser")     
    links = soup.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
         """"The above part is key because give the 
        parameters to get the information we need"""
        #Name the pdf files using the last portion of each link which are unique in this case
        filename = os.path.join(folder_location,link.a['href'].split('/')[-1]+str(".pdf"))
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link.a['href'])).content)

In [13]:
# This is a try to know what information it can be extracted and what we need to get a dataset.

archivos2018_fondospropios = []

for page in range(1,5):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start=' 
    response = requests.get(url+str((page - 1) * 15))
    
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        institucion = link.find('span').text.split('-')[-1]
        file_name = link.find('span').text
        url = link.a['href']   
       
        
        print((f'''
        Nombre institución: {institucion}
        Nombre resolucion: {file_name}
        link de descarga: {url}
        '''))


        Nombre institución: MINSA 
        Nombre resolucion: RIA-CGR-063-18-AEES-MINSA 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/7558-ria-cgr-063-18-aees-minsa
        

        Nombre institución: ANA 
        Nombre resolucion: RIA-CGR-064-18-AFYC-ANA 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/7559-ria-cgr-064-18-afyc-ana
        

        Nombre institución: VillaSandino 
        Nombre resolucion: RIA-CGR-098-18-AEES-AM-VillaSandino 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8927-ria-cgr-098-18-aees-am-villasandino
        

        Nombre institución: Nandasmo 
        Nombre resolucion: RIA-CGR-099-18-AEES-AM-Nandasmo 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8928-ria-cgr-099-18-aees-am-nandasmo
        

        Nombre institución: Camoapa 
        Nombre resolucion: RIA-CGR-100-18-AFYC-AM-Camoapa 
        link de descarga: /index.php/reposi


        Nombre institución: ENACAL 
        Nombre resolucion: RIA-CGR-1307-18-AFYC-ENACAL 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8960-ria-cgr-1307-18-afyc-enacal
        

        Nombre institución: INETER 
        Nombre resolucion: RIA-CGR-1308-18-AFYC-INETER 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8961-ria-cgr-1308-18-afyc-ineter
        

        Nombre institución: Telpaneca 
        Nombre resolucion: RIA-CGR-1309-18-AFYC-Telpaneca 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8962-ria-cgr-1309-18-afyc-telpaneca
        

        Nombre institución: Larreynaga 
        Nombre resolucion: RIA-CGR-1310-18-AFYC-AM-Larreynaga 
        link de descarga: /index.php/repositorio/send/77-cgr-fondos-propios/8963-ria-cgr-1310-18-afyc-am-larreynaga
        

        Nombre institución: Potosi 
        Nombre resolucion: RIA-CGR-1311-18-AFYC-AM-Potosi 
        link de descarga: /index.

In [20]:
""" THIS IS TO GET A DATASET OF THE INFORMATION USEFUL TO KNOW 
HOW MANY AUDIT REPORTS RESOLUTION THE CGR PUBLISH.
ADAPTING THE CODE TO EXTRACT THE PDF FILES, IT GET THE INFORMAITION """

archivos2018_fondospropios = []

for page in range(1,11):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/77-cgr-fondos-propios?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        try:
            res_fp_2018 = {
            'institucion' : link.find('span').text.split('-')[-1],
            'file_name' : link.find('span').text,
            'url' : link.a['href']
            }
            archivos2018_fondospropios.append(res_fp_2018)
        except:
            pass
    
print(len(archivos2018_fondospropios))

137


In [21]:
# NOW THAT THE INFORMATION WAS EXTRACTED AS A DICTIONARY, IT CAN BE CONVERTED TO DATAFRAME USING PANDAS

df = pd.DataFrame(archivos2018_fondospropios)
df

,institucion,file_name,url
0,MINSA,RIA-CGR-063-18-AEES-MINSA,/index.php/repositorio/send/77-cgr-fondos-prop...
1,ANA,RIA-CGR-064-18-AFYC-ANA,/index.php/repositorio/send/77-cgr-fondos-prop...
2,VillaSandino,RIA-CGR-098-18-AEES-AM-VillaSandino,/index.php/repositorio/send/77-cgr-fondos-prop...
3,Nandasmo,RIA-CGR-099-18-AEES-AM-Nandasmo,/index.php/repositorio/send/77-cgr-fondos-prop...
4,Camoapa,RIA-CGR-100-18-AFYC-AM-Camoapa,/index.php/repositorio/send/77-cgr-fondos-prop...
...,...,...,...
132,SantoTomas,RIA-CGR-970-18-AFYC-AM-SantoTomas,/index.php/repositorio/send/77-cgr-fondos-prop...
133,SantoDomingo,RIA-CGR-971-18-AFYC-AM-SantoDomingo,/index.php/repositorio/send/77-cgr-fondos-prop...
134,Cardenas,RIA-CGR-972-18-AFYC-AM-Cardenas,/index.php/repositorio/send/77-cgr-fondos-prop...
135,RioBlanco,RIA-CGR-973-18-AFYC-AM-RioBlanco,/index.php/repositorio/send/77-cgr-fondos-prop...


In [ ]:
# SAVE THE DATAFRAME

df.to_csv('CGR_fondospropios2018.csv')

**The next step is repeating the same codes for every year and audit unit as appear in the website section, using each of the link for these sections**

In [31]:
archivos2018_fondospropios2 = []

path = '/Users/leona/Documents/Python/Contraloria/2018'
files = os.listdir(path)

for root, directories, files in os.walk(path, topdown=False):
	for name in files:
                try:
                    res_fp_2018 = {
                    'institucion' : os.path.join(root, name).split('-')[-1],
                    'file_name' : os.path.join(root, name).split('/')[-1],
                    'path' : os.path.join(root, name)
                    }
                    archivos2018_fondospropios2.append(res_fp_2018)
                except:
                    pass



In [32]:
df2 = pd.DataFrame(archivos2018_fondospropios2)
df2

,institucion,file_name,path
0,villasandino.pdf,8973-ria-cgr-1457-18-afyc-am-villasandino.pdf,/Users/leona/Documents/Python/Contraloria/2018...
1,minsa.pdf,7558-ria-cgr-063-18-aees-minsa.pdf,/Users/leona/Documents/Python/Contraloria/2018...
2,elalmendro.pdf,8987-ria-cgr-1492-18-afyc-am-elalmendro.pdf,/Users/leona/Documents/Python/Contraloria/2018...
3,mific.pdf,7839-ria-cgr-626-18-afyc-mific.pdf,/Users/leona/Documents/Python/Contraloria/2018...
4,coerco.pdf,8935-ria-cgr-112-18-afyc-coerco.pdf,/Users/leona/Documents/Python/Contraloria/2018...
...,...,...,...
133,cusmapa.pdf,8086-ria-cgr-908-18-afyc-am-cusmapa.pdf,/Users/leona/Documents/Python/Contraloria/2018...
134,yalaguina.pdf,8080-ria-cgr-826-18-afyc-am-yalaguina.pdf,/Users/leona/Documents/Python/Contraloria/2018...
135,cusmapa.pdf,8088-ria-cgr-954-18-afyc-am-cusmapa.pdf,/Users/leona/Documents/Python/Contraloria/2018...
136,camoapa.pdf,7843-ria-cgr-673-18-aees-am-camoapa.pdf,/Users/leona/Documents/Python/Contraloria/2018...


In [160]:
uni_fp18=df2.institucion.unique().tolist()
sorted(uni_fp18)

['/Users/leona/Documents/Python/Contraloria/2018/.DS_Store',
 'acoyapa.pdf',
 'ana.pdf',
 'asambleanacional.pdf',
 'belen.pdf',
 'bicu.pdf',
 'bluefields.pdf',
 'boaco.pdf',
 'bocanadepaiwas.pdf',
 'bonanza.pdf',
 'buenosaires.pdf',
 'camoapa.pdf',
 'cardenas.pdf',
 'catarina.pdf',
 'chichigalpa.pdf',
 'chinandega.pdf',
 'cnu.pdf',
 'coerco.pdf',
 'corinto.pdf',
 'cornisland.pdf',
 'craccn.pdf',
 'cusmapa.pdf',
 'dgi.pdf',
 'dgspn.pdf',
 'dipilto.pdf',
 'diria.pdf',
 'diriomo.pdf',
 'elalmendro.pdf',
 'elcastillo.pdf',
 'elcoral.pdf',
 'elcua.pdf',
 'eljicaro.pdf',
 'elrealejo.pdf',
 'elrosario.pdf',
 'eltumaladalia.pdf',
 'elviejo.pdf',
 'enabas.pdf',
 'enacal.pdf',
 'esquipulas.pdf',
 'esteli.pdf',
 'fise.pdf',
 'fomav.pdf',
 'graccn.pdf',
 'granada.pdf',
 'hospitalnuevoamanecer.pdf',
 'inaa.pdf',
 'ineter.pdf',
 'inide.pdf',
 'inifom.pdf',
 'inpesca.pdf',
 'intur.pdf',
 'invur.pdf',
 'ipsa.pdf',
 'jinotepe.pdf',
 'juigalpa.pdf',
 'kukarahill.pdf',
 'laconcepcion.pdf',
 'lagunadeperl

In [34]:
df2.to_csv('CGR_fondospropios2018_path_intern.csv')

In [120]:
archivos2018_UAI = []

for page in range(1,11):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/79-uai?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        try:
            res_uai_2018 = {
            'institucion' : link.find('span').text.split('-')[-1],
            'file_name' : link.find('span').text,
            'url' : link.a['href']
            }
            archivos2018_UAI.append(res_uai_2018)
        except:
            pass
    
print(len(archivos2018_UAI))

150


In [121]:
audit_2018_uai = pd.DataFrame(archivos2018_UAI)
audit_2018_uai['institucion'] = audit_2018_uai.institucion.str.strip()
audit_2018_uai

,institucion,file_name,url
0,ASAMBLEA,RIA-UAI-001-18-AEES-ASAMBLEA,/index.php/repositorio/send/79-uai/7570-ria-ua...
1,ASAMBLEA,RIA-UAI-002-18-AEES-ASAMBLEA,/index.php/repositorio/send/79-uai/7571-ria-ua...
2,INSS,RIA-UAI-003-18-AEES-INSS,/index.php/repositorio/send/79-uai/7572-ria-ua...
3,DGA,RIA-UAI-004-18-AEES-DGA,/index.php/repositorio/send/79-uai/7573-ria-ua...
4,MINED,RIA-UAI-005-18-AEES-MINED,/index.php/repositorio/send/79-uai/7574-ria-ua...
...,...,...,...
145,Somoto,RIA-UAI-1498-18-AEES-AM-Somoto,/index.php/repositorio/send/79-uai/8464-ria-ua...
146,INTUR,RIA-UAI-1499-18-AFYC-INTUR,/index.php/repositorio/send/79-uai/8465-ria-ua...
147,SEPRES,RIA-UAI-150-18-AEES-SEPRES,/index.php/repositorio/send/79-uai/8785-ria-ua...
148,ENACAL,RIA-UAI-1500-18-AEES-ENACAL,/index.php/repositorio/send/79-uai/8466-ria-ua...


In [122]:
unique_inst_uai18=audit_2018_uai.institucion.unique().tolist()
sorted(unique_inst_uai18)

['ANA',
 'ASAMBLEA',
 'BCN',
 'Bonanza',
 'CAM',
 'CNU',
 'CSE',
 'CSJ',
 'Chichigalpa',
 'CiudadSandino',
 'DGA',
 'DGSA',
 'EAAI',
 'ENACAL',
 'ENEL',
 'EPN',
 'ElCrucero',
 'ElViejo',
 'FISE',
 'FOMAV',
 'INAA',
 'INAFOR',
 'INATEC',
 'INC',
 'INPESCA',
 'INSS',
 'INTA',
 'INTUR',
 'IPSA',
 'ISSDHU',
 'Jinotepe',
 'LOTERIA',
 'Leon',
 'MAG',
 'MARENA',
 'MEDE',
 'MEFCCA',
 'MHCP',
 'MIFAN',
 'MIFIC',
 'MIGOB',
 'MINED',
 'MINJUVE',
 'MINREX',
 'MINSA',
 'MTI',
 'Managua',
 'Masatepe',
 'Matagalpa',
 'Mateare',
 'MinisterioPublico',
 'Nagarote',
 'Nindiri',
 'NuevaGuinea',
 'PDDH',
 'PETRONIC',
 'PGR',
 'PoliciaNacional',
 'ProNicaragua',
 'ProgramaUsuraCero',
 'SEPRES',
 'SIBOIF',
 'SINAPRED',
 'SanCarlos',
 'SanRafaeldelNorte',
 'Somoto',
 'TELCOR',
 'Tipitapa',
 'Tola',
 'UAF',
 'UNA',
 'UNI']

In [5]:
pd.options.display.max_colwidth = 200

In [129]:
print(audit_2018_uai[audit_2018_uai['institucion']=='Masatepe'] ['url'])

8      /index.php/repositorio/send/79-uai/7578-ria-uai-010-18-aees-am-masatepe
104    /index.php/repositorio/send/79-uai/8752-ria-uai-115-18-aees-am-masatepe
Name: url, dtype: object


In [40]:
# EXTRACT PDF FOR AUDITORIAS 2017

for page in range(1,12):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/61-cgr?start='
    #If there is no such folder, the script will create one automatically
    folder_location = r'/Users/leona/Documents/Python/Contraloria/2017'
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url+str((page - 1) * 15))
    soup= bs(response.text, "html.parser")     
    links = soup.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        #Name the pdf files using the last portion of each link which are unique in this case
        filename = os.path.join(folder_location,link.a['href'].split('/')[-1]+str(".pdf"))
        with open(filename, 'wb') as f:
            f.write(requests.get(urljoin(url,link.a['href'])).content)

In [49]:
# dataframe auditorias 2018 fondos propios

archivos2017_FP = []

for page in range(1,12):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/61-cgr?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        try:
            res_fp_2017 = {
            'institucion' : link.find('span').text.split('-')[-1],
            'file_name' : link.find('span').text,
            'url' : link.a['href']
            }
            archivos2017_FP.append(res_fp_2017)
        except:
            pass
    
print(len(archivos2017_FP))

165


In [50]:
fp_2017 = pd.DataFrame(archivos2017_FP)
fp_2017

,institucion,file_name,url
0,MEM,RIA-CGR-1005-17-AFYC-MEM,/index.php/repositorio/send/61-cgr/7304-ria-cg...
1,Matagalpa,RIA-CGR-1006-17-AFYC-AM-Matagalpa,/index.php/repositorio/send/61-cgr/7305-ria-cg...
2,LaConquista,RIA-CGR-1007-17-AFYC-AM-LaConquista,/index.php/repositorio/send/61-cgr/7306-ria-cg...
3,Wiwili,RIA-CGR-1008-17-AFYC-AM-Wiwili,/index.php/repositorio/send/61-cgr/7307-ria-cg...
4,Quezalguaque,RIA-CGR-1009-17-AEES-AM-Quezalguaque,/index.php/repositorio/send/61-cgr/7308-ria-cg...
...,...,...,...
160,ElTumaLaDalia,RIA-CGR-892-17-AFYC-AM-ElTumaLaDalia,/index.php/repositorio/send/61-cgr/6487-ria-cg...
161,SantaRosadelPenon,RIA-CGR-893-17-AEES-AM-SantaRosadelPenon,/index.php/repositorio/send/61-cgr/6488-ria-cg...
162,Belen,RIA-CGR-894-17-AEES-AM-Belen,/index.php/repositorio/send/61-cgr/6489-ria-cg...
163,LaConcepcion,RIA-CGR-933-17-AEES-AM-LaConcepcion,/index.php/repositorio/send/61-cgr/7295-ria-cg...


In [55]:
unique_inst=fp_2017.institucion.unique().tolist()
sorted(unique_inst)

['ANA ',
 'AlemanNicaraguense ',
 'Altagracia ',
 'Belen ',
 'Boaco ',
 'BuenosAires ',
 'CAM ',
 'CGR ',
 'CNU ',
 'CONACAFE ',
 'CONAMI ',
 'Cardenas ',
 'Catarina ',
 'CentroNacionaldeCardiologia ',
 'Chichigalpa ',
 'Chinandega ',
 'CincoPinos ',
 'CiudadAntigua ',
 'CiudadDario ',
 'CiudadSandino ',
 'Comalapa ',
 'Condega ',
 'Corinto ',
 'DGI ',
 'DGME ',
 'DGSA ',
 'Dipilto ',
 'Diria ',
 'Diriamba ',
 'Diriomo ',
 'Dolores ',
 'ENACAL ',
 'ENIMPORT ',
 'ElAlmendro ',
 'ElAyote ',
 'ElCastillo ',
 'ElCrucero ',
 'ElCua ',
 'ElJicaral ',
 'ElJicaro ',
 'ElRealejo ',
 'ElTumaLaDalia ',
 'ElViejo ',
 'Esquipulas ',
 'FISE ',
 'FOMAV ',
 'GRACCS ',
 'Granada ',
 'INAA ',
 'INAFOR ',
 'INATEC ',
 'INC ',
 'IND ',
 'INETER ',
 'INIDE ',
 'INIFOM ',
 'INPESCA ',
 'INTA ',
 'IPSA ',
 'Jalapa ',
 'Jinotega ',
 'KukraHill ',
 'LaConcepcion ',
 'LaConcordia ',
 'LaConquista ',
 'LaCruzRioGrande ',
 'LaCruzdeRioGrande ',
 'LaLibertad ',
 'LaPazCentro ',
 'LagunadePerlas ',
 'Leon ',
 'MAG 

In [159]:
fp_2017[fp_2017['institucion'] == 'SantoTomasdelNorte ']

,institucion,file_name,url
28,SantoTomasdelNorte,RIA-CGR-1179-17-AFYC-AM-SantoTomasdelNorte,/index.php/repositorio/send/61-cgr/7333-ria-cgr-1179-17-afyc-am-santotomasdelnorte


In [59]:
# AUDITORÍAS 2015

archivos2015_FP = []

for page in range(1,16):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/23-cgr?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        try:
            res_fp_2015 = {
            'institucion' : link.find('span').text.split('-')[-1],
            'file_name' : link.find('span').text,
            'url' : link.a['href']
            }
            archivos2015_FP.append(res_fp_2015)
        except:
            pass
    
print(len(archivos2015_FP))

214


In [75]:
fp_2015 = pd.DataFrame(archivos2015_FP)
fp_2015

,institucion,file_name,url
0,CONACAFE,RIA-CGR-001-15-AFYC-CONACAFE,/index.php/repositorio/send/23-cgr/1977-ria-cg...
1,CiudadSandino,RIA-CGR-002-15-AEES-AM-CiudadSandino,/index.php/repositorio/send/23-cgr/1935-ria-cg...
2,RubenDario,RIA-CGR-003-15-AEES-TN-RubenDario,/index.php/repositorio/send/23-cgr/5014-ria-cg...
3,Acoyapa,RIA-CGR-004-15-AEES-AM-Acoyapa,/index.php/repositorio/send/23-cgr/1919-ria-cg...
4,ElCrucero,RIA-CGR-005-15-AEES-AM-ElCrucero,/index.php/repositorio/send/23-cgr/1926-ria-cg...
...,...,...,...
209,Juigalpa,RIA-CGR-968-15-AEES-AM-Juigalpa,/index.php/repositorio/send/23-cgr/6435-ria-cg...
210,Corinto,RIA-CGR-969-15-AEES-AM-Corinto,/index.php/repositorio/send/23-cgr/6436-ria-cg...
211,EPN,RIA-CGR-970-15-AEES-EPN,/index.php/repositorio/send/23-cgr/1947-ria-cg...
212,INATEC,RIA-CGR-971-15-AEES-INATEC,/index.php/repositorio/send/23-cgr/2023-ria-cg...


In [77]:
unique_inst15=fp_2015.institucion.unique().tolist()
sorted(unique_inst15)

[' INVUR ',
 ' Rosita ',
 'Achuapa ',
 'Acoyapa ',
 'AldoChavarria ',
 'AlemanNicaraguense ',
 'Altagracia ',
 'AsambleaNacional ',
 'BCN ',
 'Belen ',
 'BerthaCalderon ',
 'Bluefields ',
 'Bonanza ',
 'BuenosAires ',
 'CENAO ',
 'CGR ',
 'CNU ',
 'CNZF ',
 'COERCO ',
 'CONACAFE ',
 'CONAMI ',
 'CORNAP ',
 'CORREOS ',
 'CRAAN ',
 'CSE ',
 'CSJ ',
 'Camoapa ',
 'Cardenas ',
 'Catarina ',
 'Chichigalpa ',
 'Chinandega ',
 'CincoPinos ',
 'CiudadAntigua ',
 'CiudadDario ',
 'CiudadSandino ',
 'Comalapa ',
 'Condega ',
 'Corinto ',
 'DGA ',
 'DGI ',
 'DesembocaduradeRioGrande ',
 'Diriamba ',
 'Diriomo ',
 'Dolores ',
 'EAAI ',
 'ENABAS ',
 'ENACAL ',
 'ENATREL ',
 'ENIMPORT ',
 'EPN ',
 'ElAyote ',
 'ElCastillo ',
 'ElCrucero ',
 'ElCua ',
 'ElJicaral ',
 'ElJicaro ',
 'ElRosario ',
 'ElSauce ',
 'ElTortuguero ',
 'ElTumaLaDalia ',
 'ElViejo ',
 'EscuelaOscarDaniloRosales ',
 'Esquipulas ',
 'Esteli ',
 'FISE ',
 'FOMAV ',
 'FONADEFO ',
 'FONDOPESQUERO ',
 'GobiernoTasbapounie ',
 'IDR ',

In [84]:
fp_2015[fp_2015['institucion']=='Managua ']

,institucion,file_name,url
6,Managua,RIA-CGR-105-15-AEES-AM-Managua,/index.php/repositorio/send/23-cgr/1954-ria-cg...
7,Managua,RIA-CGR-106-15-AEES-AM-Managua,/index.php/repositorio/send/23-cgr/1990-ria-cg...
109,Managua,RIA-CGR-1527-15-AFYC-UNAN-Managua,/index.php/repositorio/send/23-cgr/2040-ria-cg...
193,Managua,RIA-CGR-860-15-AEES-UNAN-Managua,/index.php/repositorio/send/23-cgr/1982-ria-cg...


In [73]:
# EXTRACT PDF FOR AUDITORIAS 2015 alcaldias

for page in range(1,16):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/23-cgr?start='
    #If there is no such folder, the script will create one automatically
    folder_location = r'/Users/leona/Documents/Python/Contraloria/2015'
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url+str((page - 1) * 15))
    soup= bs(response.text, "html.parser")     
    links = soup.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        #Name the pdf files using the last portion of each link which are unique in this case
        urls = link.a['href']
        if '-am-' in urls:
            filename = os.path.join(folder_location,urls.split('/')[-1]+str(".pdf"))
            with open(filename, 'wb') as f:
                f.write(requests.get(urljoin(url,urls)).content)

In [74]:
# EXTRACT PDF FOR AUDITORIAS 2016 alcaldias

for page in range(1,16):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/20-cgr?start='
    #If there is no such folder, the script will create one automatically
    folder_location = r'/Users/leona/Documents/Python/Contraloria/2016'
    if not os.path.exists(folder_location):os.mkdir(folder_location)

    response = requests.get(url+str((page - 1) * 15))
    soup= bs(response.text, "html.parser")     
    links = soup.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        #Name the pdf files using the last portion of each link which are unique in this case
        urls = link.a['href']
        if '-am-' in urls:
            filename = os.path.join(folder_location,urls.split('/')[-1]+str(".pdf"))
            with open(filename, 'wb') as f:
                f.write(requests.get(urljoin(url,urls)).content)

In [102]:
# DATAFRAME AUDITORIAS UAI 2015

res_2018_UAI = []

for page in range(1,89):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/24-uai?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href']
        if '-am-' in urls: 
            try:
                res_UAI_2015 = {
                'institucion' : urls.split('-')[-1],
                'file_name' : urls.split('/')[-1]+str(".pdf"),
                'url' : urls
                }
                res_2018_UAI.append(res_UAI_2015)
            except:
                pass
    
print(len(res_2018_UAI))

205


In [103]:
audit_2018_UAI = pd.DataFrame(res_2018_UAI)
audit_2018_UAI

,institucion,file_name,url
0,chichigalpa,2621-ria-uai-010-15-aees-am-chichigalpa.pdf,/index.php/repositorio/send/24-uai/2621-ria-ua...
1,ocotal,2171-ria-uai-025-15-aees-am-ocotal.pdf,/index.php/repositorio/send/24-uai/2171-ria-ua...
2,esteli,6395-ria-uai-029-15-aseg-am-esteli.pdf,/index.php/repositorio/send/24-uai/6395-ria-ua...
3,masaya,1696-ria-uai-034-15-aees-am-masaya.pdf,/index.php/repositorio/send/24-uai/1696-ria-ua...
4,esteli,2601-ria-uai-036-15-afyc-am-esteli.pdf,/index.php/repositorio/send/24-uai/2601-ria-ua...
...,...,...,...
200,chichigalpa,1300-ria-uai-938-15-aees-am-chichigalpa.pdf,/index.php/repositorio/send/24-uai/1300-ria-ua...
201,rioblanco,1271-ria-uai-939-15-aees-am-rioblanco.pdf,/index.php/repositorio/send/24-uai/1271-ria-ua...
202,elviejo,1304-ria-uai-940-15-aees-am-elviejo.pdf,/index.php/repositorio/send/24-uai/1304-ria-ua...
203,managua,1277-ria-uai-942-15-aees-am-managua.pdf,/index.php/repositorio/send/24-uai/1277-ria-ua...


In [104]:
audit_2018_UAI['institucion'] = audit_2018_UAI.institucion.str.strip()

In [105]:
unique_inst18=audit_2018_UAI.institucion.unique().tolist()
sorted(unique_inst18)

['bluefields',
 'boaco',
 'bonanza',
 'chichigalpa',
 'ciudadsandino',
 'corinto',
 'elcua',
 'elviejo',
 'esteli',
 'granada',
 'jalapa',
 'jinotega',
 'juigalpa',
 'lapaz',
 'lapazcentro',
 'leon',
 'managua',
 'masatepe',
 'masaya',
 'matagalpa',
 'mateare',
 'nagarote',
 'nandasmo',
 'nindiri',
 'nuevaguinea',
 'nuevasegovia',
 'ocotal',
 'puertocabezas',
 'rioblanco',
 'rivas',
 'sancarlos',
 'sanfranciscodecuapa',
 'sanjosedecusmapa',
 'sanjuandelsur',
 'sanrafaeldelsur',
 'sanramon',
 'siuna',
 'telpaneca',
 'ticuantepe',
 'tipitapa',
 'villaelcarmen']

In [109]:
municip = ['jalapa', 'puertocabezas', 'villaelcarmen', 'telpaneca', 'managua', 'masatepe']
busqueda = audit_2018_UAI[audit_2018_UAI['institucion'].isin(municip)]
busqueda

,institucion,file_name,url
7,masatepe,2562-ria-uai-068-15-aees-am-masatepe.pdf,/index.php/repositorio/send/24-uai/2562-ria-ua...
15,villaelcarmen,5268-ria-uai-1000-15-aees-am-villaelcarmen.pdf,/index.php/repositorio/send/24-uai/5268-ria-ua...
17,managua,6400-ria-uai-101-15-aees-am-managua.pdf,/index.php/repositorio/send/24-uai/6400-ria-ua...
20,villaelcarmen,6416-ria-uai-1018-15-aees-am-villaelcarmen.pdf,/index.php/repositorio/send/24-uai/6416-ria-ua...
21,villaelcarmen,6417-ria-uai-1019-15-aees-am-villaelcarmen.pdf,/index.php/repositorio/send/24-uai/6417-ria-ua...
22,managua,2537-ria-uai-1026-15-aees-am-managua.pdf,/index.php/repositorio/send/24-uai/2537-ria-ua...
24,telpaneca,1862-ria-uai-1028-15-aees-am-telpaneca.pdf,/index.php/repositorio/send/24-uai/1862-ria-ua...
32,villaelcarmen,5269-ria-uai-1096-15-aees-am-villaelcarmen.pdf,/index.php/repositorio/send/24-uai/5269-ria-ua...
48,managua,2205-ria-uai-1112-15-aees-am-managua.pdf,/index.php/repositorio/send/24-uai/2205-ria-ua...
49,managua,1817-ria-uai-1113-15-aees-am-managua.pdf,/index.php/repositorio/send/24-uai/1817-ria-ua...


In [110]:
busqueda.to_csv('auditsUAI_2015.csv')

In [111]:
# DATAFRAME AUDITORIAS UAI 2014

audit_2014_UAI = []

for page in range(1,51):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/31-uai?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href']
        if '-am-' in urls: 
            try:
                res_UAI_2014 = {
                'institucion' : urls.split('-')[-1],
                'file_name' : urls.split('/')[-1]+str(".pdf"),
                'url' : urls
                }
                audit_2014_UAI.append(res_UAI_2014)
            except:
                pass
    
print(len(audit_2014_UAI))

82


In [113]:
audit_2014_uai = pd.DataFrame(audit_2014_UAI)
audit_2014_uai['institucion'] = audit_2014_uai.institucion.str.strip()
audit_2014_uai

,institucion,file_name,url
0,managua,3152-ria-uai-019-14-aees-am-managua.pdf,/index.php/repositorio/send/31-uai/3152-ria-ua...
1,matagalpa,3247-ria-uai-027-14-aees-am-matagalpa.pdf,/index.php/repositorio/send/31-uai/3247-ria-ua...
2,nandasmo,3134-ria-uai-029-14-aees-am-nandasmo.pdf,/index.php/repositorio/send/31-uai/3134-ria-ua...
3,managua,3035-ria-uai-030-14-aees-am-managua.pdf,/index.php/repositorio/send/31-uai/3035-ria-ua...
4,matagalpa,3149-ria-uai-046-14-aees-am-matagalpa.pdf,/index.php/repositorio/send/31-uai/3149-ria-ua...
...,...,...,...
77,mateare,6468-ria-uai-936-14-aees-am-mateare.pdf,/index.php/repositorio/send/31-uai/6468-ria-ua...
78,villaelcarmen,5276-ria-uai-938-14-aees-am-villaelcarmen.pdf,/index.php/repositorio/send/31-uai/5276-ria-ua...
79,sancarlos,3249-ria-uai-941-14-aees-am-sancarlos.pdf,/index.php/repositorio/send/31-uai/3249-ria-ua...
80,managua,2877-ria-uai-945-14-aees-am-managua.pdf,/index.php/repositorio/send/31-uai/2877-ria-ua...


In [114]:
unique_inst14=audit_2014_uai.institucion.unique().tolist()
sorted(unique_inst14)

['bluefields',
 'chichigalpa',
 'chinandega',
 'ciudadsandino',
 'condega',
 'elviejo',
 'jinotega',
 'juigalpa',
 'lapazcentro',
 'leon',
 'managua',
 'masaya',
 'matagalpa',
 'mateare',
 'nagarote',
 'nandaime',
 'nandasmo',
 'nindiri',
 'nuevaguinea',
 'ocotal',
 'rivas',
 'sancarlos',
 'sanjuandelsur',
 'sanrafaeldelsur',
 'tipitapa',
 'villaelcarmen']

In [115]:
# DATAFRAME AUDITORIAS UAI 2017

audit_2017_UAI = []

for page in range(1,35):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/63-uai?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href']
        if '-am-' in urls: 
            try:
                res_UAI_2017 = {
                'institucion' : urls.split('-')[-1],
                'file_name' : urls.split('/')[-1]+str(".pdf"),
                'url' : urls
                }
                audit_2017_UAI.append(res_UAI_2017)
            except:
                pass
    
print(len(audit_2017_UAI))

113


In [116]:
audit_2017_uai = pd.DataFrame(audit_2017_UAI)
audit_2017_uai['institucion'] = audit_2017_uai.institucion.str.strip()
audit_2017_uai

,institucion,file_name,url
0,ciudadsandino,6065-ria-uai-001-17-aees-am-ciudadsandino.pdf,/index.php/repositorio/send/63-uai/6065-ria-ua...
1,juigalpa,6080-ria-uai-016-17-aees-am-juigalpa.pdf,/index.php/repositorio/send/63-uai/6080-ria-ua...
2,esteli,6081-ria-uai-017-17-aees-am-esteli.pdf,/index.php/repositorio/send/63-uai/6081-ria-ua...
3,eltuma,6082-ria-uai-018-17-aees-am-eltuma.pdf,/index.php/repositorio/send/63-uai/6082-ria-ua...
4,leon,6083-ria-uai-019-17-aees-am-leon.pdf,/index.php/repositorio/send/63-uai/6083-ria-ua...
...,...,...,...
108,corinto,7156-ria-uai-928-17-aees-am-corinto.pdf,/index.php/repositorio/send/63-uai/7156-ria-ua...
109,leon,7157-ria-uai-929-17-aees-am-leon.pdf,/index.php/repositorio/send/63-uai/7157-ria-ua...
110,matagalpa,7158-ria-uai-930-17-aees-am-matagalpa.pdf,/index.php/repositorio/send/63-uai/7158-ria-ua...
111,managua,7159-ria-uai-931-17-aees-am-managua.pdf,/index.php/repositorio/send/63-uai/7159-ria-ua...


In [117]:
unique_inst17=audit_2017_uai.institucion.unique().tolist()
sorted(unique_inst17)

['boaco',
 'bonanza',
 'chichigalpa',
 'chinandega',
 'ciudadsandino',
 'condega',
 'corinto',
 'csj',
 'elcrucero',
 'eltuma',
 'eltumaladalia',
 'elviejo',
 'esteli',
 'granada',
 'jinotega',
 'jinotepe',
 'juigalpa',
 'leon',
 'managua',
 'masatepe',
 'matagalpa',
 'mateare',
 'mnagua',
 'muymuy',
 'nagarote',
 'nandasmo',
 'nindiri',
 'nuevaguinea',
 'ocotal',
 'rioblanco',
 'rivas',
 'sancarlos',
 'sandionisio',
 'sanjosedecusmapa',
 'sanjuandelsur',
 'sanrafaeldelnorte',
 'sanrafaeldelsur',
 'sanramon',
 'sansebastiandeyali',
 'siuna',
 'somoto',
 'ticuantepe',
 'tipitapa',
 'tola',
 'wasalala']

In [118]:
audit_2017_uai[audit_2017_uai['institucion']=='siuna']

,institucion,file_name,url
9,siuna,6111-ria-uai-047-17-aees-am-siuna.pdf,/index.php/repositorio/send/63-uai/6111-ria-ua...
10,siuna,6112-ria-uai-048-17-aees-am-siuna.pdf,/index.php/repositorio/send/63-uai/6112-ria-ua...
82,siuna,6344-ria-uai-297-17-aees-am-siuna.pdf,/index.php/repositorio/send/63-uai/6344-ria-ua...


In [119]:
audit_2017_uai.to_excel('audits_2017_uai.xlsx')

In [147]:
# DATAFRAME AUDITORIAS UAI 2019

audit_2019_UAI = []

for page in range(1,35):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/95-uai?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href'] 
        try:
            res_UAI_2019 = {
            'institucion' : urls.split('-')[-1],
            'file_name' : urls.split('/')[-1]+str(".pdf"),
            'url' : urls
            }
            audit_2019_UAI.append(res_UAI_2019)
        except:
               pass
    
print(len(audit_2019_UAI))

510


In [6]:
pd.set_option('display.max_rows',None)

In [149]:
audit_2019_uai = pd.DataFrame(audit_2019_UAI)
audit_2019_uai.sort_values('institucion')

,institucion,file_name,url
302,acoyapa,11040-ria-uai-2037-19-acum-am-acoyapa.pdf,/index.php/repositorio/send/95-uai/11040-ria-uai-2037-19-acum-am-acoyapa
252,acoyapa,11002-ria-uai-1911-19-acum-am-acoyapa.pdf,/index.php/repositorio/send/95-uai/11002-ria-uai-1911-19-acum-am-acoyapa
303,acoyapa,11041-ria-uai-2038-19-acum-am-acoyapa.pdf,/index.php/repositorio/send/95-uai/11041-ria-uai-2038-19-acum-am-acoyapa
121,ana,10884-ria-uai-1636-19-adge-ana.pdf,/index.php/repositorio/send/95-uai/10884-ria-uai-1636-19-adge-ana
488,ana,9048-ria-uai-379-19-acum-ana.pdf,/index.php/repositorio/send/95-uai/9048-ria-uai-379-19-acum-ana
472,asamblea,9032-ria-uai-358-19-acum-asamblea.pdf,/index.php/repositorio/send/95-uai/9032-ria-uai-358-19-acum-asamblea
385,asambleanacional,9995-ria-uai-247-19-acum-asambleanacional.pdf,/index.php/repositorio/send/95-uai/9995-ria-uai-247-19-acum-asambleanacional
417,asambleanacional,10027-ria-uai-287-19-acum-asambleanacional.pdf,/index.php/repositorio/send/95-uai/10027-ria-uai-287-19-acum-asambleanacional
443,asambleanacional,10053-ria-uai-321-19-acum-asambleanacional.pdf,/index.php/repositorio/send/95-uai/10053-ria-uai-321-19-acum-asambleanacional
273,asambleanacional,11021-ria-uai-1930-19-afyc-asambleanacional.pdf,/index.php/repositorio/send/95-uai/11021-ria-uai-1930-19-afyc-asambleanacional


In [150]:
unique_inst19uai=audit_2019_uai.institucion.unique().tolist()
sorted(unique_inst19uai)

['acoyapa',
 'ana',
 'asamblea',
 'asambleanacional',
 'bcn',
 'boaco',
 'cam',
 'cgr',
 'chichigalpa',
 'chinandega',
 'cinemateca',
 'ciudadsandino',
 'cnm',
 'cnu',
 'coerco',
 'conami',
 'condega',
 'corinto',
 'cornap',
 'correos',
 'correosdenicaragua',
 'craccn',
 'cse',
 'csj',
 'czf',
 'dga',
 'dgi',
 'eaai',
 'elcrucero',
 'elcua',
 'emcose',
 'enacal',
 'enatrel',
 'enel',
 'enimex',
 'enimport',
 'epn',
 'esteli',
 'fise',
 'fomav',
 'inac',
 'inafor',
 'inatec',
 'inc',
 'ind',
 'ine',
 'inide',
 'inifom',
 'iniser',
 'inpesca',
 'inss',
 'inta',
 'intur',
 'invur',
 'ipsa',
 'issdhu',
 'jalapa',
 'jinotega',
 'juigalpa',
 'leon',
 'loterianacional',
 'mag',
 'managua',
 'marena',
 'masaya',
 'matagalpa',
 'mateare',
 'mefcca',
 'mem',
 'mhcp',
 'midef',
 'mifam',
 'mific',
 'migob',
 'mined',
 'mingob',
 'minim',
 'ministeriodelajuventud',
 'ministeriopublico',
 'minjuve',
 'minrex',
 'minsa',
 'mitrab',
 'mp',
 'mti',
 'nagarote',
 'nuevaguinea',
 'ocotal',
 'petronic',


In [154]:
audit_2017_uai[audit_2017_uai['url'].str.contains('am')]

,institucion,file_name,url
0,ciudadsandino,6065-ria-uai-001-17-aees-am-ciudadsandino.pdf,/index.php/repositorio/send/63-uai/6065-ria-uai-001-17-aees-am-ciudadsandino
1,juigalpa,6080-ria-uai-016-17-aees-am-juigalpa.pdf,/index.php/repositorio/send/63-uai/6080-ria-uai-016-17-aees-am-juigalpa
2,esteli,6081-ria-uai-017-17-aees-am-esteli.pdf,/index.php/repositorio/send/63-uai/6081-ria-uai-017-17-aees-am-esteli
3,eltuma,6082-ria-uai-018-17-aees-am-eltuma.pdf,/index.php/repositorio/send/63-uai/6082-ria-uai-018-17-aees-am-eltuma
4,leon,6083-ria-uai-019-17-aees-am-leon.pdf,/index.php/repositorio/send/63-uai/6083-ria-uai-019-17-aees-am-leon
5,nagarote,6084-ria-uai-020-17-aees-am-nagarote.pdf,/index.php/repositorio/send/63-uai/6084-ria-uai-020-17-aees-am-nagarote
6,granada,6095-ria-uai-031-17-aees-am-granada.pdf,/index.php/repositorio/send/63-uai/6095-ria-uai-031-17-aees-am-granada
7,nindiri,6097-ria-uai-033-17-aees-aees-am-nindiri.pdf,/index.php/repositorio/send/63-uai/6097-ria-uai-033-17-aees-aees-am-nindiri
8,mateare,6110-ria-uai-046-17-aees-am-mateare.pdf,/index.php/repositorio/send/63-uai/6110-ria-uai-046-17-aees-am-mateare
9,siuna,6111-ria-uai-047-17-aees-am-siuna.pdf,/index.php/repositorio/send/63-uai/6111-ria-uai-047-17-aees-am-siuna


In [155]:
# DATAFRAME AUDITORIAS FONDOS PROPIOS 2019

audit_2019_fp = []

for page in range(1,11):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/108-cgr-fondos-propios736?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href'] 
        try:
            res_FP_2019 = {
            'institucion' : urls.split('-')[-1],
            'file_name' : urls.split('/')[-1]+str(".pdf"),
            'url' : urls
            }
            audit_2019_fp.append(res_FP_2019)
        except:
               pass
    
print(len(audit_2019_fp))

147


In [158]:
fp_audit_2019 = pd.DataFrame(audit_2019_fp)
fp_audit_2019.sort_values('institucion')

,institucion,file_name,url
25,acoyapa,11117-ria-cgr-1475-19-afyc-am-acoyapa.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11117-ria-cgr-1475-19-afyc-am-acoyapa
2,asamblea,9913-ria-cgr-1147-19-afyc-asamblea.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/9913-ria-cgr-1147-19-afyc-asamblea
65,belen,11157-ria-cgr-1899-19-afyc-am-belen.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11157-ria-cgr-1899-19-afyc-am-belen
66,benemeritocuerpobomberosmanagua,10081-ria-cgr-190-19-afyc-benemeritocuerpobomberosmanagua.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/10081-ria-cgr-190-19-afyc-benemeritocuerpobomberosmanagua
64,bluefields,11156-ria-cgr-1898-19-afyc-am-bluefields.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11156-ria-cgr-1898-19-afyc-am-bluefields
7,bluefields,11099-ria-cgr-1331-19-aees-am-bluefields.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11099-ria-cgr-1331-19-aees-am-bluefields
69,boaco,11160-ria-cgr-1902-19-afyc-am-boaco.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11160-ria-cgr-1902-19-afyc-am-boaco
77,buenosaires,11168-ria-cgr-1910-19-afyc-am-buenosaires.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11168-ria-cgr-1910-19-afyc-am-buenosaires
138,catarina,9903-ria-cgr-724-19-afyc-am-catarina.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/9903-ria-cgr-724-19-afyc-am-catarina
50,cgr,11142-ria-cgr-1770-19-afyc-cgr.pdf,/index.php/repositorio/send/108-cgr-fondos-propios736/11142-ria-cgr-1770-19-afyc-cgr


In [ ]:
# DATAFRAME AUDITORIAS FONDOS PROPIOS 2021

audit_2019_fp = []

for page in range(1,11):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/108-cgr-fondos-propios736?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href'] 
        try:
            res_FP_2019 = {
            'institucion' : urls.split('-')[-1],
            'file_name' : urls.split('/')[-1]+str(".pdf"),
            'url' : urls
            }
            audit_2019_fp.append(res_FP_2019)
        except:
               pass
    
print(len(audit_2019_fp))

In [3]:
# DATAFRAME AUDITORIAS FONDOS PROPIOS 2020

audit_2020_fp = []

for page in range(1,7):
    url = 'https://www.cgr.gob.ni/index.php/repositorio/category/116-cgr-fondos-propios?start=' 
    #response = requests.get(url)
    response = requests.get(url+str((page - 1) * 15))
    content = bs(response.text, 'html.parser')
    links = content.find_all('div', attrs={'class':'row tsa_registro'})
    for link in links:
        urls = link.a['href'] 
        try:
            res_FP_2020 = {
            'institucion' : urls.split('-')[-1],
            'file_name' : urls.split('/')[-1]+str(".pdf"),
            'url' : urls
            }
            audit_2020_fp.append(res_FP_2020)
        except:
               pass
    
print(len(audit_2020_fp))

73


In [7]:
fp_audit_2020 = pd.DataFrame(audit_2020_fp)
fp_audit_2020.sort_values('institucion')

,institucion,file_name,url
70,acoyapa,12714-ria-cgr-2023-2020-acum-am-acoyapa.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12714-ria-cgr-2023-2020-acum-am-acoyapa
11,asambleanacional,12655-ria-cgr-1639-2020-afyc-asambleanacional.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12655-ria-cgr-1639-2020-afyc-asambleanacional
5,belen,12649-ria-cgr-1575-2020-afyc-am-belen.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12649-ria-cgr-1575-2020-afyc-am-belen
68,buenosaires,12712-ria-cgr-2021-2020-afyc-am-buenosaires.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12712-ria-cgr-2021-2020-afyc-am-buenosaires
33,cam,12677-ria-cgr-1810-2020-afin-cam.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12677-ria-cgr-1810-2020-afin-cam
67,catarina,12711-ria-cgr-2020-2020-afyc-am-catarina.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12711-ria-cgr-2020-2020-afyc-am-catarina
72,chichigalpa,12716-ria-cgr-2025-2020-acum-am-chichigalpa.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12716-ria-cgr-2025-2020-acum-am-chichigalpa
12,condega,12656-ria-cgr-1640-2020-afyc-am-condega.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12656-ria-cgr-1640-2020-afyc-am-condega
1,dgi,11438-ria-cgr-065-2020-aaes-dgi.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/11438-ria-cgr-065-2020-aaes-dgi
55,dgi,12699-ria-cgr-1915-2020-afyc-dgi.pdf,/index.php/repositorio/send/116-cgr-fondos-propios/12699-ria-cgr-1915-2020-afyc-dgi
